# Changer le CSV et le nom des DF

- output_(n).csv
cellule 4

- df_data_villes_clean(n)
cellules: 4, 5, 6

- df_meilleur_agent(n)
cellules: 7, 8

# Vérifier la shape de df_data_villes_clean(n) et df_meilleur_agent(n) 

il doit y avoir le même nombre de lignes

In [16]:
#pip install selenium selenium-stealth beautifulsoup4 pandas

In [17]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random


In [18]:
df_data_villes_clean3 = pd.read_csv("pays_de_la_loire3.csv", 
                                    index_col=0,
                                    dtype=str)


In [19]:
df_data_villes_clean3.head()

,#Code_commune_INSEE,Lib_MA,departement,Code_postal,Nom_de_la_commune_url,url
19667,49266,ST AUGUSTIN DES BOIS,49,49170,saint-augustin-des-bois,https://www.meilleursagents.com/prix-immobilie...
19668,49267,ST BARTHELEMY D ANJOU,49,49124,saint-barthelemy-d-anjou,https://www.meilleursagents.com/prix-immobilie...
19669,49269,ST CHRISTOPHE DU BOIS,49,49280,saint-christophe-du-bois,https://www.meilleursagents.com/prix-immobilie...
19670,49271,ST CLEMENT DE LA PLACE,49,49370,saint-clement-de-la-place,https://www.meilleursagents.com/prix-immobilie...
19671,49272,ST CLEMENT DES LEVEES,49,49350,saint-clement-des-levees,https://www.meilleursagents.com/prix-immobilie...


In [20]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Lancer le navigateur
driver.get("https://www.meilleursagents.com/prix-immobilier/")

In [21]:
# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_clean3.iterrows():
    city = row['Lib_MA']+" ("+row['Code_postal']+")"
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        # Trouver la barre de recherche
        search_box = driver.find_element(By.NAME, "q")

        # Simuler une frappe clavier, caractère par caractère
        for y in city:
            search_box.clear()
            search_box.send_keys(y)
            time.sleep(0.05)  # petite pause pour rendre la frappe réaliste

        # Valider avec la touche Entrée
        time.sleep(1)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        data_dict = {}
        if not raw_text3 :
            driver.refresh()
            time.sleep(3)
        else:
            # Ajouter la ville
            title_text = soup.find("title").get_text().strip()
            ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
            data_dict["ville"] = ville

            for ul in raw_text3:
                li_items = ul.find_all("li")
                label = li_items[0].get_text().strip().lower()
                main_value = extract_number(li_items[1].get_text())

                # Nettoyer le texte pour récupérer min et max correctement
                range_text = li_items[2].get_text()
                range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")

                # Extraire tous les nombres sous forme de float
                numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
                if "prix" in label:
                    if "prix_appartement" not in data_dict:
                        data_dict["prix_appartement"] = main_value
                        data_dict["min_appartement"] = numbers_in_range[0]
                        data_dict["max_appartement"] = numbers_in_range[1]
                    else:
                        data_dict["prix_maison"] = main_value
                        data_dict["min_maison"] = numbers_in_range[0]
                        data_dict["max_maison"] = numbers_in_range[1]
                elif "loyer" in label:
                    if "loyer_appartement" not in data_dict:
                        data_dict["loyer_appartement"] = main_value
                        data_dict["loyer_min_appartement"] = numbers_in_range[0]
                        data_dict["loyer_max_appartement"] = numbers_in_range[1]
                    else:
                        data_dict["loyer_maison"] = main_value
                        data_dict["loyer_min_maison"] = numbers_in_range[0]
                        data_dict["loyer_max_maison"] = numbers_in_range[1]

            # Ajouter l'URL pour référence
            data_dict["city"] = city
            all_data.append(data_dict)
            print(f"Datas ajoutées pour la ville : {ville}")

    except Exception as e:
        print(f"Erreur pour la ville {city}: {e}")
        error_data.append({
        "city": city,
        "erreur": str(e)})

    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(1.8, 3.6))

driver.quit()

# Convertir en DataFrame
# df_scraped = pd.DataFrame(all_data)
# df_scraped.head()

Erreur pour la ville ST AUGUSTIN DES BOIS (49170): Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="q"]"}
  (Session info: chrome=139.0.7258.139); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ee6578d5+2802741]
	GetHandleVerifier [0x0x7ff7ee3beb70+79568]
	(No symbol) [0x0x7ff7ee15c0fa]
	(No symbol) [0x0x7ff7ee1b2a96]
	(No symbol) [0x0x7ff7ee1b2d4c]
	(No symbol) [0x0x7ff7ee206017]
	(No symbol) [0x0x7ff7ee1daccf]
	(No symbol) [0x0x7ff7ee202e24]
	(No symbol) [0x0x7ff7ee1daa63]
	(No symbol) [0x0x7ff7ee1a3c91]
	(No symbol) [0x0x7ff7ee1a4a23]
	GetHandleVerifier [0x0x7ff7ee682ced+2979917]
	GetHandleVerifier [0x0x7ff7ee67d0f3+2956371]
	GetHandleVerifier [0x0x7ff7ee69acbd+3078173]
	GetHandleVerifier [0x0x7ff7ee3d836e+184014]
	GetHandleVerifier [0x0x7ff7ee3e024f+216495]
	GetHandleVerifier [0x0x7ff7ee3c70c4+113700]
	

KeyboardInterrupt: 

In [22]:
driver.quit()


In [23]:
df_meilleur_agent3 = pd.DataFrame(all_data)

In [ ]:
df_meilleur_agent3.sample(5)    

,ville,prix_appartement,min_appartement,max_appartement,prix_maison,min_maison,max_maison,loyer_appartement,loyer_min_appartement,loyer_max_appartement,loyer_maison,loyer_min_maison,loyer_max_maison,city
107,Saint Hilaire du Bois,2589.0,2031.0,3074.0,1876.0,750.0,2794.0,11.4,8.2,19.8,9.9,6.6,12.6,YZERNAY (49360)
99,Saint Hilaire du Bois,2246.0,1124.0,3333.0,1843.0,737.0,2918.0,11.3,7.6,18.1,9.8,7.1,12.9,VIHIERS (49310)
35,: le prix du m² partout en France - Août 2025,2244.0,1824.0,2760.0,1237.0,495.0,2468.0,11.3,8.2,19.8,9.7,6.3,12.6,ST PAUL DU BOIS (49310)
106,Saint Hilaire du Bois,1614.0,1138.0,3229.0,1972.0,842.0,3285.0,11.0,7.7,19.5,9.3,5.4,13.0,VIVY (49680)
9,: le prix du m² partout en France - Août 2025,2448.0,1125.0,3782.0,2162.0,865.0,3487.0,10.9,7.8,15.7,11.4,9.0,16.3,ST AUBIN DE LUIGNE (49190)
